# **rectorch**: basic concepts

This tutorial will show examples of how to train and test a model. The tutorial will focus on baseline models but the concepts are almost the same in the case of more advanced models.

## Preliminaries

### Dataset download
For the purposes of this tutorial we download the *movielens 1M* dataset. As the name suggests, this dataset contains roughly one million (5 stars) ratings about movies. For more details, please refer to the official web page https://grouplens.org/datasets/movielens/1m/.

In [ ]:
%cd /content/
!wget http://files.grouplens.org/datasets/movielens/ml-1m.zip
!unzip ml-1m.zip
!rm ml-1m.zip

/content
--2020-09-14 14:17:53--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip’

ml-1m.zip           100%[===================>]   5.64M  13.8MB/s    in 0.4s    

2020-09-14 14:17:54 (13.8 MB/s) - ‘ml-1m.zip’ saved [5917549/5917549]

Archive:  ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         


### **rectorch** installation

NOTE: in this version of the tutorial we load the *dev* version from [github](https://github.com/makgyver/rectorch).

In [ ]:
%cd /content/
!git clone -b dev https://github.com/makgyver/rectorch.git
%cd rectorch
!pip install -r requirements.txt

/content
Cloning into 'rectorch'...
remote: Enumerating objects: 55, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 1705 (delta 17), reused 35 (delta 17), pack-reused 1650
Receiving objects: 100% (1705/1705), 3.22 MiB | 20.87 MiB/s, done.
Resolving deltas: 100% (1133/1133), done.
/content/rectorch
     |████████████████████████████████| 10.5MB 6.3MB/s 
ERROR: google-colab 1.0.0 has requirement pandas~=1.0.0; python_version >= "3.0", but you'll have pandas 1.1.2 which is incompatible.
  Found existing installation: pandas 1.0.5
    Uninstalling pandas-1.0.5:
      Successfully uninstalled pandas-1.0.5


### Data loading and splitting

In [ ]:
cfg_data = {
    "processing": {
        "data_path": "../ml-1m/ratings.dat",
        "threshold": 3.5,
        "separator": "::",
        "header": None,
        "u_min": 2,
        "i_min": 0
    },
    "splitting": {
        "split_type": "vertical",
        "sort_by": None,
        "seed": 98765,
        "shuffle": True,
        "valid_size": 100,
        "test_size": 100,
        "test_prop": 0.2
    }
}

In [ ]:
from rectorch.data import DataProcessing
dataset = DataProcessing(cfg_data).process_and_split()
dataset

[14:19:28-140920]  Reading raw data file ../ml-1m/ratings.dat.
[14:19:32-140920]  NumExpr defaulting to 2 threads.
[14:19:33-140920]  Thresholded 424928 ratings.
[14:19:33-140920]  Applying filtering.
[14:19:33-140920]  Filtered 1 ratings.
[14:19:33-140920]  Shuffling data.
[14:19:33-140920]  Calculating splits.
[14:19:33-140920]  Creating validation and test set.
[14:19:33-140920]  Skipped 2 ratings in validation set.
[14:19:33-140920]  Skipped 3 ratings in test set.


Dataset(n_users=6037, n_items=3528, n_ratings=575275)

For more details about how to load, process and splitting the dataset, please refer to the tutorial [rectorch_data_tutorial.ipynb](https://colab.research.google.com/drive/1gKgMllkYlvvBqh7q6WmmSvtfAOTz7tFh#scrollTo=Cwi1HjgJ-T7Z).

## Standard pipeline

**rectorch** tries to make it the coders life easy by fixing a standard pipeline when it comes to training, and testing a recommender system.

The pipeline follows this steps:

* **Dataset creation**: seen above and in [this tutorial](https://colab.research.google.com/drive/1gKgMllkYlvvBqh7q6WmmSvtfAOTz7tFh#scrollTo=Cwi1HjgJ-T7Z);
* **Sampler creation**: a sampler is an object that "prepares" the dataset for the recommender system. Some models can handle different data formats, while others require a specific sampler to work properly;
* **Model initialization**: the creation of the model object with its own hyper-parameters;
* **Training**: the training of the model which may need the sampler to handle the data;
* **Evaluation**: the testing of the model that requires the sampler since it uses the `predict` method of the model to get the prediction that are compared with the ground truth.

### Sampler creation: rectorch.samplers

A sampler is a convenient way to handle the dataset. It works like an interface between the dataset (which encapsulates a `pandas.Dataframe`) and the model. Standard samplers provided by **rectorch** handle different types of data or, in the case of neural model, they provide different ways of creating mini-batches.

In the case of the baselines, we will use only the `rectorch.samplers.ArrayDummySampler` and `rectorch.samplers.SparseDummySampler`. The former handle the dataset as a `numpy.ndarray`, while the second as a `scipy.sparse.csr_matrix`.

In [ ]:
from rectorch.samplers import ArrayDummySampler
array_sampler = ArrayDummySampler(dataset, mode="train")

The `mode` of a sampler indicates its current state, that is which part of the dataset is handling. In this case, the training set ("train") since we are going to train the models.

### Model initialization: Random recommender

A random recommender is simply a system that recommends random items to users. The only useful parameter to initialize the model is the number of items.

In [ ]:
from rectorch.models.baseline import Random
rnd = Random(dataset.n_items)

The training procedure is actually empty. However, for completeness lets call it.

### Training

In [ ]:
rnd.train(array_sampler) #useless call for this recommender

### Evaluation

Evaluating the performance of a model is really easy in **rectorch**. The simplest way is to call the `rectorch.evaluation.evaluate` function passing the model, the sampler and the list of metrics. To date, **rectorch** supports the following metrics:
* hit@k
* ndcg@k
* recall@k
* ap@k
* mrr@k
* auc

When the metric has the parameter `k` it is automatically inferred from the provided string. Thus, giving "ndcg@100" is interpreted as `ndcg@k` with `k=100`. 

In [ ]:
from rectorch.evaluation import evaluate
array_sampler.test()
results = evaluate(rnd, array_sampler, ["ndcg@10", "recall@10"])

`results` is a dictionary where the keys are the tested metrics, and values (numpy arrays) are the scores for each user with that specific metric. Since usually it is convenient to compute the overall average of these values, **rectorch** provide a function (i.e.,`rectorch.utils.collect_results`) that does it. It returns, for each metric, the mean and stardard deviation as a pair of `float` values.

In [ ]:
from rectorch.utils import collect_results
collect_results(results)

{'ndcg@10': (0.006845139397573328, 0.026022437770306373),
 'recall@10': (0.0071111111111111115, 0.025940101944458293)}

**NOTE**: before calling the `evaluate` function, we switched the sampler mode to `test`. This is a good practice since the evaluation can be also done in the validation set (not in the training!!). However, if the sampler is in `training` mode, when passed to the `evaluate` it will be automatically switched to `test` but a warning will appear.

In [ ]:
array_sampler.train() #train mode to see the warning
results = evaluate(pop, array_sampler, ["ndcg@10", "recall@10"])
collect_results(results)

[14:58:33-140920]  Sampler must be in valid or test mode. Froced switch to test mode!


{'ndcg@10': (0.09573019268352317, 0.11177790052280144),
 'recall@10': (0.09690079365079365, 0.10916368351154157)}

## Other baseline models

### Popularity based recommender

Popularity based recommender suggests the most popular items.

In [ ]:
from rectorch.models.baseline import Popularity
pop = Popularity(dataset.n_items)
pop.train(array_sampler)

In [ ]:
array_sampler.test()
results = evaluate(pop, array_sampler, ["ndcg@10", "recall@10"])
collect_results(results)

### CF-KOMD

CF-KOMD is a kernel-based method proposed by Polato et al. [[1](https://www.sciencedirect.com/science/article/abs/pii/S0925231218300900), [2](https://www.sciencedirect.com/science/article/abs/pii/S0925231217307592), [3](https://www.elen.ucl.ac.be/Proceedings/esann/esannpdf/es2016-111.pdf)].
The method is based on the concept of maximal margin (like in SVM) to compute the ranking between items. The optimization problem producing the ranking can be computed in parallel since for each user a different (and rather small) optimization problem is created.

In [ ]:
from rectorch.models.baseline import CF_KOMD
cfkomd = CF_KOMD(ker_fun="linear", disj_degree=1, lam=0.1)
cfkomd.train(array_sampler, only_test=True) #creates the model only for test users

In [ ]:
array_sampler.test()
results = evaluate(cfkomd, array_sampler, ["ndcg@10", "recall@10"])
collect_results(results)

{'ndcg@10': (0.10585826119158127, 0.176156407032027),
 'recall@10': (0.10102380952380952, 0.16248748599992546)}

### SLIM

SLIM: Sparse Linear Methods for Top-N Recommender Systems.

The SLIM [4](https://ieeexplore.ieee.org/document/6137254) model can be presented as
$$\tilde{\mathbf{A}} = \mathbf{A}\mathbf{W}$$
where $\mathbf{A}$ is the rating matrix, $\mathbf{W}$ is an $n \times n$ sparse matrix of aggregation coefficients, and where each row of $\tilde{\mathbf{A}}$ represents the recommendation scores on all items for a user.

The column of $\mathbf{W}$ are learned independently by solving the following optimization problem:

$$ \operatorname{min}_{\mathbf{w}_{j}} \frac{1}{2} \| \mathbf{a}_{j} - A \mathbf{w}_{j} \|_{2}^{2} + \frac{\beta}{2} \left\| \mathbf{w}_{j} \right\|_{2}^{2}+\lambda \left\|\mathbf{w}_{j}\right\|_{1} $$

subject to
$$\mathbf{w}_{j} \geq \mathbf{0}, \: w_{j, j}=0$$

where ``l1_reg`` is $\lambda$ and ``l2_reg`` is $\beta$.

SLIM requires a different sampler because it handles sparse data.

In [ ]:
from rectorch.models.baseline import SLIM
from rectorch.samplers import SparseDummySampler

sparse_sampler = SparseDummySampler(dataset, mode="train")
slim = SLIM(l1_reg=0.001, l2_reg=0.001)
slim.train(sparse_sampler)

[15:15:28-140920]  | item 352/3528 | ms/user 23.86 |
[15:15:36-140920]  | item 704/3528 | ms/user 22.76 |
[15:15:43-140920]  | item 1056/3528 | ms/user 21.68 |
[15:15:51-140920]  | item 1408/3528 | ms/user 20.51 |
[15:15:57-140920]  | item 1760/3528 | ms/user 19.23 |
[15:16:04-140920]  | item 2112/3528 | ms/user 18.80 |
[15:16:10-140920]  | item 2464/3528 | ms/user 17.36 |
[15:16:16-140920]  | item 2816/3528 | ms/user 15.15 |
[15:16:20-140920]  | item 3168/3528 | ms/user 11.84 |
[15:16:22-140920]  | item 3520/3528 | ms/user 6.69 |
[15:16:22-140920]  | training complete | total training time 62.70 s |


In [ ]:
sparse_sampler.test()
results = evaluate(slim, sparse_sampler, ["ndcg@10", "recall@10"])
collect_results(results)

{'ndcg@10': (0.32459041141786826, 0.24383642039019238),
 'recall@10': (0.31967460317460317, 0.22155421976980716)}